** Pavel Smirnov @ github.com/pysmirnov **

That's how we are to calculate resonator's characteristics:

In [3]:
# scale factors
def freq_m(freq):
    freq *= 1e3; return freq;
def cap_m(cap):
    cap /= 1e9; return cap;

def inductance(freq,cap):
    return 1/(freq_m(freq)**2 * cap_m(cap))

def quality_factor(freq,cap,res):
    return 1/(freq_m(freq) * cap_m(cap) * res)

def reactive_res(freq,cap):
    return 1/(freq_m(freq)*cap_m(cap))

def resistance_ind(rho,q):
    return rho/q
                                         
def resistance_smax(rho):
    return rho*1e-3 # tan(1e-3) \approx 1e-3, yeah
                                         
def current(e_voltage,res_s):
    return e_voltage/res_s

def resistance_sum(r,r_s,r_l):
    return r+r_s+r_l
    #return (ind/cap_m(cap)*res_l) * sqrt( 1 + (res_l/(freq_m(freq)*ind)**2 ))

And now gonna check out what's going on in our circuit...

In [14]:
import numpy as np
from math import pi
data = np.loadtxt('data_exp.tsv', delimiter='\t')

R_L = []
L = []

out = open("data.tsv",'w')
for n in range(7):
    cap = data[n][0]
    freq = data[n][1]*2*math.pi
    u = data[n][2]
    e = data[n][3]
    r = 3.5 # Ohm
    
    ind = inductance(freq,cap)
    q = quality_factor(freq,cap,r)
    rho = reactive_res(freq,cap)
    r_s = resistance_smax(rho)
    r_l = resistance_ind(rho,q)
    r_sum = resistance_sum(r,r_s,r_l)
    i = current(e,r_s)
    
    R_L.append(r_l)
    L.append(ind)
    
    out.write(str(n+1)+'\t')
    for i in range(4):
        out.write(str(data[n][i])+'\t')
    # weird printf-like syntax...
    out.write("%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\t%0.2f\n" % (ind*1e6,q,rho,r_sum,r_s,r_l,i*1e3))
    
out.write("R_L = "+str(np.average(R_L))+' Ом'+'\n')
out.write("L = "+str(np.average(L)*1e6)+' мкГн'+'\n')
    
out.close()